In [1]:
from dotenv import load_dotenv
load_dotenv('./.env')
import json
import openai
import os
from dspy import (
    OpenAI,
    OllamaLocal,
    HFModel,
    Predict,
    Signature,
    ChainOfThought,
    settings,
    predict,
    context
)
import warnings
warnings.filterwarnings('ignore')

/Users/hritik.raj/DSPy/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
openai.api_key= os.environ['OPENAI_API_KEY']

In [5]:
from rich import print

In [6]:
turbo = OpenAI(model='gpt-3.5-turbo')
settings.configure(lm=turbo)

In [7]:
signatures={"QuestionAnswering":"question -> answer",
            "SentimentClassification":"sentence -> sentiment",
            "Summarization":["document -> summary",
                             "text -> gist",
                             "long_context -> tldr"],
            "RAG":"context, question -> answer",
            "Multi-Choice":"question, choices -> reasoning, selection"}

In [8]:
signatures['QuestionAnswering']

'question -> answer'

In [9]:
# Predict class uses a signature of this form 
# only a single Predict class is required
qa_pred = Predict('question -> answer')


In [10]:
print(qa_pred)

Predict(StringSignature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Answer:', 'desc': '${answer}'})
))

In [11]:
sum_cot = ChainOfThought('document -> summary')

In [12]:
print(sum_cot)

self = Predict(StringSignature(document -> rationale, summary
    instructions='Given the fields `document`, produce the fields `summary`.'
    document = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Document:', 'desc': '${document}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by 
step in order to", 'desc': '${produce the summary}. We ...', '__dspy_field_type': 'output'})
    summary = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Summary:', 'desc': '${summary}'})
))

In [13]:
from dspy import(
    InputField,
    OutputField
)

class MultiClass(Signature):
    """Classift the given data into Address, Humans Name, Location, Building, Amount"""
    sentence = InputField(desc="data to be specified")
    data_type = OutputField(desc="falls in one of categories")



In [14]:
pred_class = Predict(MultiClass)
print(pred_class)

Predict(MultiClass(sentence -> data_type
    instructions='Classift the given data into Address, Humans Name, Location, Building, Amount'
    sentence = Field(annotation=str required=True json_schema_extra={'desc': 'data to be specified', 
'__dspy_field_type': 'input', 'prefix': 'Sentence:'})
    data_type = Field(annotation=str required=True json_schema_extra={'desc': 'falls in one of categories', 
'__dspy_field_type': 'output', 'prefix': 'Data Type:'})
))

In [15]:
get_json_data = Predict('required_data -> json_output')

prompt="provide me one eg of Address, Location, Human Name, building name and amount"

with context(lm=turbo):
    response = get_json_data(required_data=prompt)
    print("Response by OPENAI: ", response.json_output)

 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Response by OPENAI:  {
  "Address": "123 Main Street",
  "Location": "New York City",
  "Human Name": "John Doe",
  "Building Name": "Empire State Building",
  "Amount": "$100"
}

In [65]:
data= json.loads(response.json_output)

NameError: name 'json' is not defined

In [16]:
from dspy import ProgramOfThought

In [17]:
generate_answer_signature= Signature("question -> answer")

#pass the sig to ProgramOfThought Module
pot = ProgramOfThought(generate_answer_signature)

#call the program of thought Module on a prompt
question='Hritik has 5 apples. He buys 3 more apples. How many apples does he have now?'
result = pot(question=question)

print(f"Question:{question}")
print(f"Answer:{result}")


Question:Hritik has 5 apples. He buys 3 more apples. How many apples does he have now?

Answer:Prediction(
    rationale='produce the answer. We first define the initial number of apples Hritik has as 5 and the number of 
apples he buys as 3. By adding these two values together, we can calculate the total number of apples Hritik has 
now.',
    answer='Hritik has 8 apples now.'
)

In [21]:
turbo.inspect_history(n=6)




Given the fields `required_data`, produce the fields `json_output`.

---

Follow the following format.

Required Data: ${required_data}
Json Output: ${json_output}

---

Required Data: provide me one eg of Address, Location, Human Name, building name and amount
Json Output: {
  "Address": "123 Main Street",
  "Location": "New York City",
  "Human Name": "John Doe",
  "Building Name": "Empire State Building",
  "Amount": "$100"
}





You will be given `question` and you will respond with `answer`.
Generating executable Python code that programmatically computes the correct `answer`.
After you're done with the computation, make sure the last line in your code evaluates to the correct value for `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the generated_code}. We ...
Code: python code that answers the question

---

Question: Hritik has 5 apples. He buys 3 more apples. How many apples does he have now?
Rea

'\n\n\nGiven the fields `required_data`, produce the fields `json_output`.\n\n---\n\nFollow the following format.\n\nRequired Data: ${required_data}\nJson Output: ${json_output}\n\n---\n\nRequired Data: provide me one eg of Address, Location, Human Name, building name and amount\nJson Output:\x1b {\n  "Address": "123 Main Street",\n  "Location": "New York City",\n  "Human Name": "John Doe",\n  "Building Name": "Empire State Building",\n  "Amount": "$100"\n}\x1b\n\n\n\n\n\nYou will be given `question` and you will respond with `answer`.\nGenerating executable Python code that programmatically computes the correct `answer`.\nAfter you\'re done with the computation, make sure the last line in your code evaluates to the correct value for `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let\'s think step by step in order to ${produce the generated_code}. We ...\nCode: python code that answers the question\n\n---\n\nQuestion: Hritik has 5 apples. He buys 